In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../data/csgo_round_snapshots.csv')
df.head

<bound method NDFrame.head of         time_left  ct_score  t_score       map  bomb_planted  ct_health  \
0          175.00       0.0      0.0  de_dust2         False      500.0   
1          156.03       0.0      0.0  de_dust2         False      500.0   
2           96.03       0.0      0.0  de_dust2         False      391.0   
3           76.03       0.0      0.0  de_dust2         False      391.0   
4          174.97       1.0      0.0  de_dust2         False      500.0   
...           ...       ...      ...       ...           ...        ...   
122405      15.41      11.0     14.0  de_train          True      200.0   
122406     174.93      11.0     15.0  de_train         False      500.0   
122407     114.93      11.0     15.0  de_train         False      500.0   
122408      94.93      11.0     15.0  de_train         False      500.0   
122409      74.93      11.0     15.0  de_train         False      375.0   

        t_health  ct_armor  t_armor  ct_money  ...  t_grenade_flashba

In [4]:
from src.utils.data_prep import *

df_without_unused_weapons = del_unused_weapons(df)
df_one_hot_col = one_hot_col(df_without_unused_weapons,'map')
df_final = name2bin(df_one_hot_col,'round_winner')

unused columns:['t_weapon_aug', 'ct_weapon_bizon', 't_weapon_bizon', 'ct_weapon_elite', 't_weapon_elite', 't_weapon_famas', 'ct_weapon_g3sg1', 't_weapon_g3sg1', 'ct_weapon_glock', 'ct_weapon_m249', 't_weapon_m249', 't_weapon_m4a1s', 'ct_weapon_mac10', 'ct_weapon_mag7', 't_weapon_mag7', 'ct_weapon_mp5sd', 't_weapon_mp5sd', 'ct_weapon_mp7', 't_weapon_mp7', 't_weapon_mp9', 'ct_weapon_negev', 't_weapon_negev', 'ct_weapon_nova', 't_weapon_nova', 'ct_weapon_p90', 't_weapon_p90', 'ct_weapon_r8revolver', 't_weapon_r8revolver', 'ct_weapon_sawedoff', 't_weapon_sawedoff', 'ct_weapon_scar20', 't_weapon_scar20', 'ct_weapon_xm1014', 't_weapon_xm1014', 't_weapon_fiveseven', 't_weapon_p2000', 'ct_weapon_tec9']


E:\MAJEUR IA\DEEP LEARNING\DL_final_project\src\utils\data_prep.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded = df_encoded.replace({True:1, False:0})
E:\MAJEUR IA\DEEP LEARNING\DL_final_project\src\utils\data_prep.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({cat_1:1, cat_2:0})


In [5]:
print(df_final)

        time_left  ct_score  t_score  bomb_planted  ct_health  t_health  \
0          175.00       0.0      0.0             0      500.0     500.0   
1          156.03       0.0      0.0             0      500.0     500.0   
2           96.03       0.0      0.0             0      391.0     400.0   
3           76.03       0.0      0.0             0      391.0     400.0   
4          174.97       1.0      0.0             0      500.0     500.0   
...           ...       ...      ...           ...        ...       ...   
122405      15.41      11.0     14.0             1      200.0     242.0   
122406     174.93      11.0     15.0             0      500.0     500.0   
122407     114.93      11.0     15.0             0      500.0     500.0   
122408      94.93      11.0     15.0             0      500.0     500.0   
122409      74.93      11.0     15.0             0      375.0     479.0   

        ct_armor  t_armor  ct_money  t_money  ...  t_grenade_decoygrenade  \
0            0.0      

In [6]:
y_cat = df_final['round_winner']
y_reg = df_final[['ct_money','ct_health']]
X = df_final.drop(columns=['round_winner','ct_money','ct_health'])

X_train, X_test, y_train_reg, y_test_reg, y_train_cat, y_test_cat = train_test_split(
    X, y_reg, y_cat, test_size=0.2, random_state=42
)

scaler_X = preprocessing.StandardScaler()
scaler_y = preprocessing.StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_reg_scaled = scaler_y.fit_transform(y_train_reg)
y_test_reg_scaled = scaler_y.transform(y_test_reg)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

cols_reg = ['ct_money', 'ct_health']
y_train_reg_scaled = pd.DataFrame(y_train_reg_scaled, columns=cols_reg)
y_test_reg_scaled = pd.DataFrame(y_test_reg_scaled, columns=cols_reg)

X_train_scaled.to_csv('../data/processed/X_train.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test.csv', index=False)

y_train_reg_scaled.to_csv('../data/processed/y_train_reg_scaled.csv', index=False)
y_test_reg_scaled.to_csv('../data/processed/y_test_reg_scaled.csv', index=False)

y_train_cat.to_csv('../data/processed/y_train_cat.csv', index=False)
y_test_cat.to_csv('../data/processed/y_test_cat.csv', index=False)